In [1]:
 # Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd

In [2]:
csv_file_path = ("../Car price data/prius_data.csv")

In [3]:
# Using Splinter Scraping data
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
# URL of page to be scraped
url = 'https://losangeles.craigslist.org/search/cto?min_price=2000&auto_make_model=prius&min_auto_year=2008&max_auto_year=2018&min_auto_miles=0&max_auto_miles=1000000&condition=10&condition=20&condition=30&condition=40'

In [4]:
# Scrape the data based on post info
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html,"html.parser")
next_page = soup.find("a",class_ = "button next")["href"]

titles = []
prices = []
links = []
car_data = []

while next_page:
    print(" - - - - - - - - - - - Scraping New Page Starts!! - - - - - - - - - ")
    html = browser.html
    soup = BeautifulSoup(html,"html.parser")
    results = soup.find_all('li', class_="result-row")
    
    
    
    
    for result in results:
    # Error handling
        try:
            # Identify and return title of listing
            title = result.find('a', class_="result-title").text
            # Identify and return price of listing
            price = result.a.span.text
            # Identify and return link to listing
            link = result.a['href']

            # Print results only if title, price, and link are available
            if (title and price and link):
                titles.append(title)
                prices.append(price)
                links.append(link)
                
                post_link = result.a["href"]
                response2 = requests.get(post_link)
                soup2 = BeautifulSoup(response2.text,"html.parser")
                attributes = soup2.find_all("p",class_="attrgroup")
                attribute_list = attributes[1].find_all("span")
                attr_dic = {}
                
                try:
                
                    tag = attributes[0].find("span").find("b").text.split(" ")
                
                    yearlist = [str(num) for num in list(range(1900, 2020))]
                    for word in tag:
                        if word in yearlist:
                            year = word
                            for attr in attribute_list:
                                try:
                                    key,value = attr.text.split(":")
                                    attr_dic[key] = value
                                    attr_dic["Year"] = year
                                except ValueError:
                                    pass
                            if word not in tag:
                                attr_dic["Year"] = "NA"
                    
                    
                except:
                    attr_dic["Year"] = "NA"
                
    
                car_data.append(attr_dic)
                
                
        except AttributeError as e:
            print(e)
    try:
        browser.click_link_by_partial_text('next')
        next_page = soup.find("a",class_ = "button next")["href"]
    except:
        print("- - - - - - - - - This is the last page we can find!!!- - - - - -")
        next_page = False

 - - - - - - - - - - - Scraping New Page Starts!! - - - - - - - - - 
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
 - - - - - - - - - - - Scraping New Page Starts!! - - - - - - - - - 
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
- - - - - - - - - This is the last page we can find!!!- - - - - -


In [5]:
# Store info as dataframe
info_df = pd.DataFrame({
    "Info" : titles,
    "Price" : prices,
    "links" : links
})

In [6]:
# Show full dataframe
pd.set_option('display.max_colwidth', -1)
# Merge two dataframe
car_df = pd.DataFrame(car_data)
car_df["Title"] = info_df["Info"]
car_df["Price"] = info_df["Price"]
car_df["Link"] = info_df["links"]
car_df["Make"] = "Toyota"
car_df["Model"] = "Prius"
car_df = car_df[['Title','Make', 'Model','Year', 'condition', 'cylinders', 'drive', 'fuel', 'odometer',
       'paint color', 'size', 'title status', 'transmission', 'type', 
       'Price', 'VIN', 'Link']]
car_df.head(50)

,Title,Make,Model,Year,condition,cylinders,drive,fuel,odometer,paint color,size,title status,transmission,type,Price,VIN,Link
0,"2009 Toyota Pirus IV, 112000 miles",Toyota,Prius,2009,excellent,4 cylinders,fwd,hybrid,112000,white,compact,clean,automatic,sedan,$7800,JTDKB20U797858488,https://losangeles.craigslist.org/lgb/cto/d/2009-toyota-pirus-ivmiles/6703088577.html
1,2009 Toyota Prius clean title clean carfax camera 178k,Toyota,Prius,2009,excellent,4 cylinders,fwd,hybrid,178925,NaN,compact,clean,automatic,hatchback,$4400,NaN,https://losangeles.craigslist.org/wst/cto/d/2009-toyota-prius-clean-title/6710612940.html
2,"2008 TOYOTA PRIUS -- 81,000 MILES -",Toyota,Prius,2008,excellent,4 cylinders,fwd,hybrid,81000,green,compact,clean,automatic,hatchback,$10500,jtdkb20u583368766,https://losangeles.craigslist.org/wst/cto/d/2008-toyota-priusmiles/6697992822.html
3,** 2011 Toyota Prius **,Toyota,Prius,2011,excellent,4 cylinders,fwd,hybrid,83600,white,mid-size,clean,automatic,hatchback,$9900,NaN,https://losangeles.craigslist.org/sgv/cto/d/2011-toyota-prius/6710538316.html
4,2010 Toyota Prius,Toyota,Prius,2010,excellent,NaN,NaN,hybrid,113000,NaN,NaN,clean,automatic,NaN,$7400,NaN,https://losangeles.craigslist.org/lac/cto/d/2010-toyota-prius/6710549852.html
5,2013 Toyota prius IV low 87k miles clean title,Toyota,Prius,2013,excellent,NaN,NaN,gas,87277,NaN,NaN,clean,automatic,NaN,$12900,NaN,https://losangeles.craigslist.org/lgb/cto/d/2013-toyota-prius-iv-low-87k/6694760217.html
6,Black 2017 Toyota Prius 2 Eco Hatchback 4D,Toyota,Prius,2017,new,4 cylinders,fwd,hybrid,5300,black,compact,clean,automatic,hatchback,$22500,JTDKBRFU9H3048267,https://losangeles.craigslist.org/sfv/cto/d/black-2017-toyota-prius-2-eco/6708405876.html
7,2016 Toyota Prius 3 Touring,Toyota,Prius,2016,like new,4 cylinders,fwd,hybrid,58500,white,full-size,clean,automatic,sedan,$20900,JTDKARFU9G3016694,https://losangeles.craigslist.org/lac/cto/d/2016-toyota-prius-3-touring/6708272360.html
8,2009 Toyota Prius Hatchback *31 Service Recs* CLEAN TITLE,Toyota,Prius,2009,good,4 cylinders,NaN,hybrid,196000,NaN,NaN,clean,automatic,hatchback,$3995,NaN,https://losangeles.craigslist.org/lgb/cto/d/2009-toyota-prius-hatchback/6710321306.html
9,2009 Toyota Prius,Toyota,Prius,2009,like new,4 cylinders,fwd,hybrid,149000,white,full-size,clean,automatic,hatchback,$5800,NaN,https://losangeles.craigslist.org/sfv/cto/d/2009-toyota-prius/6710313554.html


In [7]:
# Out put as CSV file
car_df.to_csv(csv_file_path)